base.py 

In [1]:
import torchvision
from torchvision import transforms
#from .wrapper import CacheClassLabel, MyDataset, AppendName
from torch.utils.data import  DataLoader
import PIL
from torch.utils.data import Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split


import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import pandas as pd

""" THIS PART BELONGS TO THE WRAPPER IN DATA"""
import torch.utils.data as data
class AppendName(data.Dataset):
    """
    A dataset wrapper that also return the name of the dataset/task
    """
    def __init__(self, dataset, name):
        super(AppendName,self).__init__()
        self.dataset = dataset
        self.name = name

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img,target = self.dataset[index]
        return img, target, self.name
    
class MyDataset(Dataset):
    def __init__(self, data, targets,root, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform
        self.root = root

    def __getitem__(self, index):
        x = self.data[index]
        y = int(self.targets[index])

        if self.transform:
            x = Image.fromarray(self.data[index].astype(np.uint8).transpose(1,2,0))
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

class MyDataset_BP4D(Dataset):
    def __init__(self, data, targets, root, transform=None):
        self.data = data
        self.targets = targets  # Assuming targets is a dictionary
        self.transform = transform
        self.root = root

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Assuming self.data is a dictionary with file names as keys
        filename = list(self.data.keys())[index]
        image = self.data[filename]
        target = self.targets[filename]

        # Convert NumPy array to PIL Image
        image = Image.fromarray(image.astype(np.uint8).transpose(1, 2, 0))

        if self.transform is not None:
            image = self.transform(image)

        return image, target


# class DISFADataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.samples = []

#         # Load all samples into a list
#         for img_folder in sorted(os.listdir(os.path.join(root_dir, 'Images'))):
#             images = [Image.open(os.path.join(root_dir, 'Images', img_folder, f'{i:03d}.jpg')).convert('RGB') for i in range(len(os.listdir(os.path.join(root_dir, 'Images', img_folder))))]

#             labels = {}
#             labels_folder = os.path.join(root_dir, 'Labels', img_folder)
#             for au_file in os.listdir(labels_folder):
#                 au, _ = os.path.splitext(au_file)
#                 au_data = torch.from_numpy(
#                     np.loadtxt(os.path.join(labels_folder, au_file), dtype=np.float32, skiprows=2, usecols=1)
#                 ).long()
#                 labels[au] = au_data

#             self.samples.append((images, labels))

#     def __len__(self):
#         return len(self.samples)

#     def __getitem__(self, idx):
#         images, labels = self.samples[idx]

#         if self.transform:
#             images = [self.transform(img) for img in images]

#         images = torch.stack(images, dim=0)

#         return images, labels
    

# class DISFADataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.samples = []

#         # Load all samples into a list
#         for img_folder in sorted(os.listdir(os.path.join(root_dir, 'Images'))):
#             images = [Image.open(os.path.join(root_dir, 'Images', img_folder, f'{i:03d}.jpg')).convert('RGB') for i in range(len(os.listdir(os.path.join(root_dir, 'Images', img_folder))))]

#             labels = {}
#             labels_folder = os.path.join(root_dir, 'Labels', img_folder)
#             for au_file in os.listdir(labels_folder):
#                 au, _ = os.path.splitext(au_file)
#                 au_data = torch.from_numpy(
#                     np.loadtxt(os.path.join(labels_folder, au_file), dtype=np.float32, skiprows=2, usecols=1)
#                 ).long()
#                 labels[au] = au_data

#             self.samples.append((images, labels))

#     def __len__(self):
#         return len(self.samples)

#     def __getitem__(self, idx):
#         images, labels = self.samples[idx]

#         if self.transform:
#             images = [self.transform(img) for img in images]

#         # Concatenate images along the batch dimension
#         images = torch.stack(images, dim=0)

#         # Concatenate labels along the batch dimension
#         labels_tensor = torch.stack(list(labels.values()), dim=1)

#         return images, labels_tensor



import pickle
import torchvision
from torchvision import transforms
#from .wrapper import CacheClassLabel, MyDataset, AppendName
from torch.utils.data import  DataLoader
from PIL import Image
import pickle
from os import listdir
from os.path import isfile, join
import numpy as np
import torch


def load_image1(filename):
    """read an image and convert it to numpy array and returns it"""
    img = Image.open(filename)
    img.load()
    img = img.resize((32,32)) 
    img = np.transpose(img,(2,0,1))
    data = np.asarray(img, dtype="int32")
    return data    
    


def BP4DDataset(root_dir=r'D:\\BP4D\\',transform=False):


    BP4D_IMAGES = root_dir + 'images_crop'
    # Images paths and Occurrence labels used to create a pickle file.
    #BP4D_OCCURENCE = root_dir + 'aus_bp4d_occurrence.pkl'
    BP4D_OCCURENCE = root_dir + "Filtered_BP4D_OCCURENCE.pkl"




    # Use the same transformation for both training and validation datasets

    mypath = BP4D_IMAGES

    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

    file = BP4D_OCCURENCE
    with open(file, "rb") as f:
        label = pickle.load(f)

    datax = {}

    au_list = ["1", "2", "4", "6", "7", "10", "12", "14", "15", "17", "23", "24"]    
    for i,f in enumerate(onlyfiles):
        datax[f[:-4]] = load_image1(mypath + "/" + f)


    from sklearn.model_selection import train_test_split

    # Assuming datax is a dictionary with keys and label is a dictionary with corresponding labels

    # Extract keys from datax
    keys = list(datax.keys())

    # Split the keys into training and validation sets
    train_keys, val_keys = train_test_split(keys, test_size=0.2, random_state=42)

    # Use the split keys to extract corresponding data and labels
    train_x = {key: datax[key] for key in train_keys}
    train_y = {key: label[key] for key in train_keys}
    val_x = {key: datax[key] for key in val_keys}
    val_y = {key: label[key] for key in val_keys}

    # Now use train_x, train_y, val_x, and val_y in your MyDataset_BP4D constructor
    train_datasets = MyDataset_BP4D(train_x, train_y, "data", transform)
    val_datasets = MyDataset_BP4D(val_x, val_y, "data", transform)


    return train_datasets, val_datasets
        


class RAFDBDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        image_dir = os.path.join(root_dir, 'Image\\aligned\\aligned')
        self.image_dir = image_dir
        self.annotation_dir = os.path.join(root_dir, 'Annotation/manual/manual')
        self.emotion_labels_path = os.path.join(root_dir, 'EmoLabel/list_patition_label.txt')
        self.transform = transform
        self.data = []

        emotion_labels_path = os.path.join(root_dir, 'EmoLabel/list_patition_label.txt')
        emotion_labels = pd.read_csv(emotion_labels_path, sep=' ', names=['image_name', 'emotion_label'], header=None)
        emotion_labels['image_name'] = emotion_labels['image_name'].str.replace('.jpg', '_aligned.jpg')
        self.emotion_labels = emotion_labels

        # Loop through the dataset to load images and labels
        for idx in range(len(emotion_labels)):
            image_name = emotion_labels.iloc[idx, 0]
            image_path = os.path.join(image_dir, image_name)
            image = Image.open(image_path).convert('RGB')

            if transform:
                image = transform(image)

            # Load emotion label
            emotion_label = emotion_labels.iloc[idx, 1]

            self.data.append((image, emotion_label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, emotion_label = self.data[idx]

        return image, emotion_label
    
import os
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision.transforms import Compose, ToTensor, Normalize
from PIL import Image
import pandas as pd

class ArousalValenceDataset(Dataset):
    def __init__(self, data_root, transform=None):
        self.data_root = data_root
        self.transform = transform
        self.samples = []

        # Load dataset
        self._load_data()

    def _load_data(self):
        images_dir = os.path.join(self.data_root, 'images')
        annotations_dir = os.path.join(self.data_root, r'3rd ABAW Annotations\Third ABAW Annotations\VA_Estimation_Challenge\Train_Set')

        for folder_name in os.listdir(images_dir):
            images_folder_path = os.path.join(images_dir, folder_name)
            if not os.path.isdir(images_folder_path):
                continue

            annotation_file = f'{folder_name}.txt'
            annotation_path = os.path.join(annotations_dir, annotation_file)
            if not os.path.isfile(annotation_path):
                continue

            # Read arousal and valence values
            annotations = pd.read_csv(annotation_path)

            for i, row in annotations.iterrows():
                image_file = f'{(i+1):05}.jpg'
                image_path = os.path.join(images_folder_path, image_file)

                if not os.path.isfile(image_path):
                    continue

                self.samples.append((image_path, row['valence'], row['arousal']))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, valence, arousal = self.samples[idx]

        image = Image.open(image_path)
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor([valence, arousal], dtype=torch.float)





def load_image(filename):
    """read an image and convert it to numpy array and returns it"""
    img = Image.open(filename)
    img.load()
    #img = img.resize((45,55)) 
    img = np.transpose(img,(2,0,1))
    data = np.asarray(img, dtype="int32")
    return data


def ConFAR(transform):
    # this function reads RAF-DB data and it separates the samples given category. Output as dictionary. (dict["1"] = male samples, dict["2"] = female samples for gender ..)
    
    transform = transform
    
    rafdbdata = r'D:\RAFDB\RAF-DB\RAF-DB-20231017T081935Z-001\RAF-DB\basic'
    disfadata = r"C:\Users\KARAN\Desktop\Cambridge\LibreFace\data\DISFA"
    affwild2= r"C:\Users\KARAN\Desktop\Cambridge\DATASETS\AffWild2"
    
    train_datasets = {} # 1: RAF-DB train 2: DISFA Tran and 3: Affwild2 Train
    val_datasets = {}

    task_output_space = {"AU": 12, "FER": 7}#, "AV": 2}

    n_tasks = len(task_output_space)

    # for key in task_output_space:
    #     if key == 'FER':
    #         all_data = RAFDBDataset(transform=train_transform, root_dir=rafdbdata) # update this for FER 
    #         train_dataset, val_dataset = train_test_split(all_data, test_size=0.2, random_state=42)
    #         print("Loaded RAFDB for FER")

    #     elif key =='AU':
    #         all_data = DISFADataset(transform=train_transform, root_dir=disfadata) # update this for AU 
    #         train_dataset, val_dataset = train_test_split(all_data, test_size=0.2, random_state=42)
    #         print("Loaded DISFA for AU")

    #     train_datasets[key] = AppendName(train_dataset, key)
    #     val_datasets[key] = AppendName(val_dataset, key)

    for key in task_output_space:
        if key == 'FER':
            all_data = RAFDBDataset(transform=transform, root_dir=rafdbdata) # update this for FER 
            train_dataset, val_dataset = train_test_split(all_data, test_size=0.2, random_state=42)
            print("Loaded RAFDB for FER")

        elif key =='AV':
            full_data = ArousalValenceDataset(data_root=affwild2, transform=transform)
            train_dataset, val_dataset = train_test_split(full_data, test_size=0.2, random_state=42)
            print("Loaded Affwild2 for AV")

        elif key =='AU':
            train_dataset, val_dataset = BP4DDataset(transform=transform) # update this for AU 
            print("Loaded BP4D for AU")

        train_datasets[key] = AppendName(train_dataset, key)
        val_datasets[key] = AppendName(val_dataset, key)
        
    

    return train_datasets, val_datasets, task_output_space



# Create a folder with 3 different sub-folders
# 1st folder will be the FER dataset (RAFDB)
## Train and Test
### 6 clases under each

# 2nd folder will be the AU dataset (DISFA)
## Train and Test
### Images and labels

# 3rd folder will be the AV dataset (AffWild2)
## Train and Test
### Images and labels


d:\anaconda\envs\TORCHGPU\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'D:\anaconda\envs\TORCHGPU\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# transform = transforms.Compose([
#     transforms.Resize((100, 100)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# a,b,c = ConFAR(transform)

In [3]:
import os
import sys
import argparse
import torch
import numpy as np
from random import shuffle
from collections import OrderedDict
import agents
#import dataloaders.base
#from dataloaders.base import ConFAR


def run(args):
    if not os.path.exists('outputs'):
        os.mkdir('outputs')

    transform = transforms.Compose([
        transforms.Resize((100, 100)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    #train_dataset_splits, val_dataset_splits, task_output_space = RafDB_perm(args.category, args.train_aug)
    train_dataset_splits, val_dataset_splits, task_output_space = ConFAR(transform)

    # Print the shape of a sample from the training datasets
    for key, dataset in train_dataset_splits.items():
        print(f"Training dataset shape for task {key}: {len(dataset)}")
        sample_x, sample_y, _ = dataset[0]  # Assuming the first sample is representative
        print(f"Training dataset sample shape for task {key} - x: {sample_x.shape}, y: {sample_y.shape}")

    # Print the shape of a sample from the validation datasets
    for key, dataset in val_dataset_splits.items():
        print(f"Validation dataset shape for task {key}: {len(dataset)}")
        sample_x, sample_y, _ = dataset[0]  # Assuming the first sample is representative
        print(f"Validation dataset sample shape for task {key} - x: {sample_x.shape}, y: {sample_y.shape}")


    # Decide split ordering
    task_names = sorted(list(task_output_space.keys())) #SORT THIS HERE SUXYHDGBFN
    #task_names = list(task_output_space.keys()) #SORT THIS HERE SUXYHDGBFN
    print('Task order:',task_names)
    
    agent_config = {'tasks':task_names,'lr': args.lr, 'momentum': args.momentum, 'weight_decay': args.weight_decay,'schedule': args.schedule,
                            'model_type':args.model_type, 'model_name': args.model_name, 'model_weights':args.model_weights,
                            'out_dim':{'FER':args.force_out_dim_fer,'AU':args.force_out_dim_au} ,
                            'optimizer':args.optimizer,
                            'print_freq':args.print_freq, 'gpuid': args.gpuid,
                            'reg_coef':args.reg_coef}
    agent = agents.__dict__[args.agent_type].__dict__[args.agent_name](agent_config)
    print(agent.model)
    print('# of parameters:',agent.count_parameter())
    acc_table = OrderedDict()
    f1_table = OrderedDict()
    

    for i in range(len(task_names)):
        
        print(len(train_dataset_splits[task_names[i]]))
        train_name = task_names[i]
        print('======================',train_name,'=======================')
        train_loader = torch.utils.data.DataLoader(train_dataset_splits[train_name],
                                                    batch_size=args.batch_size, shuffle=True, num_workers=0)
        val_loader = torch.utils.data.DataLoader(val_dataset_splits[train_name],
                                                    batch_size=args.batch_size, shuffle=False, num_workers=0)

        if args.incremental_class:
            agent.add_valid_output_dim(task_output_space[train_name])

        # Learn
        print("cats = ",train_loader, val_loader,train_name,sep="  ")
        agent.learn_batch(train_loader, val_loader,i)

        # Evaluate
        acc_table[train_name] = {} #OrderedDict()
        f1_table[train_name] = {} #OrderedDict()
        for j in range(i+1):
            val_name = task_names[j]
            print('validation split name:', val_name)
            val_data = val_dataset_splits[val_name] if not args.eval_on_train_set else train_dataset_splits[val_name]
            val_loader = torch.utils.data.DataLoader(val_data,
                                                        batch_size=args.batch_size, shuffle=False,
                                                        num_workers=0)
            acc_table[train_name][val_name], f1_table[train_name][val_name] = agent.validation(val_loader,val_name)
            
            """ table's format :
            [FER: FER
            AU:  FER,  AU]
            """

    return acc_table, task_names, f1_table

def get_args(argv):
    # This function prepares the variables shared across demo.py
    parser = argparse.ArgumentParser()
    parser.add_argument('--gpuid', nargs="+", type=int, default=[0],
                        help="The list of gpuid, ex:--gpuid 3 1. Negative value means cpu-only")
    parser.add_argument('--model_type', type=str, default='mlp', help="The type (mlp|lenet|vgg|resnet) of backbone network")
    parser.add_argument('--model_name', type=str, default='MLP', help="The name of actual model for the backbone")
    parser.add_argument('--force_out_dim_fer', type=int, default=7, help="Set 0 to let the task decide the required output dimension")
    parser.add_argument('--force_out_dim_au', type=int, default=12, help="Set 0 to let the task decide the required output dimension")
    #parser.add_argument('--force_out_dim', type=int, default=7, help="Set 0 to let the task decide the required output dimension")
    parser.add_argument('--agent_type', type=str, default='default', help="The type (filename) of agent")
    parser.add_argument('--agent_name', type=str, default='NormalNN', help="The class name of agent")
    parser.add_argument('--optimizer', type=str, default='SGD', help="SGD|Adam|RMSprop|amsgrad|Adadelta|Adagrad|Adamax ...")
    parser.add_argument('--dataroot', type=str, default='data', help="The root folder of dataset or downloaded data")
    parser.add_argument('--dataset', type=str, default='MNIST', help="MNIST(default)|CIFAR10|CIFAR100")
    parser.add_argument('--n_permutation', type=int, default=0, help="Enable permuted tests when >0")
    parser.add_argument('--first_split_size', type=int, default=2)
    parser.add_argument('--other_split_size', type=int, default=2)
    parser.add_argument('--no_class_remap', dest='no_class_remap', default=False, action='store_true',
                        help="Avoid the dataset with a subset of classes doing the remapping. Ex: [2,5,6 ...] -> [0,1,2 ...]")
    parser.add_argument('--train_aug', dest='train_aug', default=False, action='store_true',
                        help="Allow data augmentation during training")
    parser.add_argument('--rand_split', dest='rand_split', default=False, action='store_true',
                        help="Randomize the classes in splits")
    parser.add_argument('--rand_split_order', dest='rand_split_order', default=False, action='store_true',
                        help="Randomize the order of splits")
    parser.add_argument('--workers', type=int, default=3, help="#Thread for dataloader")
    parser.add_argument('--batch_size', type=int, default=100)
    parser.add_argument('--lr', type=float, default=0.01, help="Learning rate")
    parser.add_argument('--momentum', type=float, default=0)
    parser.add_argument('--weight_decay', type=float, default=0)
    parser.add_argument('--schedule', nargs="+", type=int, default=[2],
                        help="The list of epoch numbers to reduce learning rate by factor of 0.1. Last number is the end epoch")
    parser.add_argument('--print_freq', type=float, default=100, help="Print the log at every x iteration")
    parser.add_argument('--model_weights', type=str, default=None,
                        help="The path to the file for the model weights (*.pth).")
    parser.add_argument('--reg_coef', nargs="+", type=float, default=[0.], help="The coefficient for regularization. Larger means less plasilicity. Give a list for hyperparameter search.")
    parser.add_argument('--eval_on_train_set', dest='eval_on_train_set', default=False, action='store_true',
                        help="Force the evaluation on train set")
    parser.add_argument('--offline_training', dest='offline_training', default=False, action='store_true',
                        help="Non-incremental learning by make all data available in one batch. For measuring the upperbound performance.")
    parser.add_argument('--repeat', type=int, default=1, help="Repeat the experiment N times")
    parser.add_argument('--incremental_class', dest='incremental_class', default=True, action='store_true',
                        help="The number of output node in the single-headed model increases along with new categories.")
    parser.add_argument('--category', type=str, default='gender', help="The Category (gender, race, age)") 
    args = parser.parse_args(argv)
    return args



In [4]:
if __name__ == '__main__':

    # args = get_args(sys.argv[1:])  UNCOMMENT IN FINAL CODE
    args = get_args([
    '--gpuid', '0',
    '--repeat', '1',
    '--optimizer', 'Adam',
    '--no_class_remap',
    # '--force_out_dim', '7',
    '--force_out_dim_fer', '7',
    '--force_out_dim_au', '12',
    '--schedule', '1',
    '--batch_size', '24',
    '--model_type', 'custom_cnn',
    '--model_name', 'Net',
    '--agent_type', 'customization',
    #'--agent_name', 'EWC',
    '--lr', '0.0001',
    '--reg_coef', '1', '10', '100',
    '--train_aug'
    ])
    reg_coef_list = args.reg_coef

    avg_final_acc = {}

    # The for loops over hyper-paramerters or repeats
    for reg_coef in reg_coef_list:
        args.reg_coef = reg_coef
        avg_final_acc[reg_coef] = np.zeros(args.repeat)
        for r in range(args.repeat):

            # Run the experiment
            acc_table, task_names, f1_table = run(args)
            print("The Accuracy table is ",acc_table)
            
            name = args.category if args.train_aug == False else args.category + "_augmented"
            if not os.path.exists(f'results/{args.category}'):
                os.makedirs(f'results/{args.category}')
            with open('results/' + args.category + '/'  + name + '.txt', 'a') as f:
                f.write("\n" + str(acc_table) + "f1: " +  str(f1_table) + " repeat:" + str(r) + " reg_coef: " + str(reg_coef)  + " " +  str(args.agent_name))
            # Calculate average performance across tasks
            # Customize this part for a different performance metric

            avg_acc_history = [0] * len(task_names)
            for i in range(len(task_names)):
                train_name = task_names[i]
                cls_acc_sum = 0
                for j in range(i + 1):
                    val_name = task_names[j]
                    cls_acc_sum += acc_table[train_name][val_name]
                avg_acc_history[i] = cls_acc_sum / (i + 1)                    #err here
                print('Task', train_name, 'average acc:', avg_acc_history[i])

            # Gather the final avg accuracy
            avg_final_acc[reg_coef][r] = avg_acc_history[-1]

            # Print the summary so far
            print('===Summary of experiment repeats:',r+1,'/',args.repeat,'===')
            print('The regularization coefficient:', args.reg_coef)
            print('The last avg acc of all repeats:', avg_final_acc[reg_coef])
            print('mean:', avg_final_acc[reg_coef].mean(), 'std:', avg_final_acc[reg_coef].std())
    for reg_coef,v in avg_final_acc.items():
        print('reg_coef:', reg_coef,'mean:', avg_final_acc[reg_coef].mean(), 'std:', avg_final_acc[reg_coef].std())


Loaded BP4D for AU
Loaded RAFDB for FER
Training dataset shape for task AU: 932
Training dataset sample shape for task AU - x: torch.Size([3, 100, 100]), y: (12,)
Training dataset shape for task FER: 1600
Training dataset sample shape for task FER - x: torch.Size([3, 100, 100]), y: ()
Validation dataset shape for task AU: 233
Validation dataset sample shape for task AU - x: torch.Size([3, 100, 100]), y: (12,)
Validation dataset shape for task FER: 400
Validation dataset sample shape for task FER - x: torch.Size([3, 100, 100]), y: ()
Task order: ['AU', 'FER']
Net(
  (layer1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=(2, 2), stride=(

d:\anaconda\envs\TORCHGPU\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
d:\anaconda\envs\TORCHGPU\lib\site-packages\torch\optim\lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https

[0/39]	1.7383 (1.7383)	0.0636 (0.0636)	11.363 (11.363)	29.17 (29.17)
[38/39]	0.7669 (0.9068)	0.0188 (0.0212)	0.000 (1.048)	30.00 (32.19)
 * Train Acc 32.189
f1 score: 0.26904761904761904
 * Val Acc 33.047, Total time 3.50
validation split name: AU
f1 score: 0.26904761904761904
 * Val Acc 33.047, Total time 3.49
1600
====================== FER =======================
Incremental class: Old valid output dimension: 12
Incremental class: New Valid output dimension: 19
cats =   <torch.utils.data.dataloader.DataLoader object at 0x000001836026BE20>  <torch.utils.data.dataloader.DataLoader object at 0x000001836026BDC0>  FER
Epoch:0
LR: 0.0001
Itr		Time		  Data		  Loss		Acc


d:\anaconda\envs\TORCHGPU\lib\site-packages\torch\optim\lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[0/67]	0.9946 (0.9946)	0.0025 (0.0025)	3.014 (3.014)	0.00 (0.00)
[66/67]	0.5762 (0.8575)	0.0010 (0.0021)	1.472 (1.504)	50.00 (46.06)
 * Train Acc 46.062
f1 score: 0.4157006369426751
 * Val Acc 57.000, Total time 5.49
validation split name: AU
f1 score: 0.2843915343915344
 * Val Acc 28.755, Total time 3.41
validation split name: FER
f1 score: 0.4157006369426751
 * Val Acc 57.000, Total time 5.92
The Accuracy table is  OrderedDict([('AU', {'AU': 33.047210300429185}), ('FER', {'AU': 28.755364806866954, 'FER': 57.0})])
Task AU average acc: 33.047210300429185
Task FER average acc: 42.877682403433475
===Summary of experiment repeats: 1 / 1 ===
The regularization coefficient: 1.0
The last avg acc of all repeats: [42.8776824]
mean: 42.877682403433475 std: 0.0
Loaded BP4D for AU
Loaded RAFDB for FER
Training dataset shape for task AU: 932
Training dataset sample shape for task AU - x: torch.Size([3, 100, 100]), y: (12,)
Training dataset shape for task FER: 1600
Training dataset sample shape for

...

[0/67]	0.7426 (0.7426)	0.0021 (0.0021)	3.807 (3.807)	0.00 (0.00)
[66/67]	0.5771 (0.8512)	0.0016 (0.0022)	1.097 (1.552)	62.50 (47.69)
 * Train Acc 47.688
f1 score: 0.41686009538950713
 * Val Acc 57.250, Total time 5.72
validation split name: AU
f1 score: 0.5307539682539683
 * Val Acc 37.339, Total time 3.61
validation split name: FER
f1 score: 0.41686009538950713
 * Val Acc 57.250, Total time 5.95
The Accuracy table is  OrderedDict([('AU', {'AU': 33.047210300429185}), ('FER', {'AU': 37.33905579399141, 'FER': 57.25})])
Task AU average acc: 33.047210300429185
Task FER average acc: 47.2945278969957
===Summary of experiment repeats: 1 / 1 ===
The regularization coefficient: 10.0
The last avg acc of all repeats: [47.2945279]
mean: 47.2945278969957 std: 0.0
Loaded BP4D for AU
Loaded RAFDB for FER
Training dataset shape for task AU: 932
Training dataset sample shape for task AU - x: torch.Size([3, 100, 100]), y: (12,)
Training dataset shape for task FER: 1600
Training dataset sample shape for 

d:\anaconda\envs\TORCHGPU\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
d:\anaconda\envs\TORCHGPU\lib\site-packages\torch\optim\lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https

[0/39]	1.3814 (1.3814)	0.0196 (0.0196)	9.035 (9.035)	41.67 (41.67)
[38/39]	2.0239 (1.5005)	0.0467 (0.0339)	0.000 (0.638)	30.00 (33.05)
 * Train Acc 33.047
f1 score: 0.26904761904761904
 * Val Acc 33.047, Total time 6.61
validation split name: AU
f1 score: 0.26904761904761904
 * Val Acc 33.047, Total time 6.87
1600
====================== FER =======================
Incremental class: Old valid output dimension: 12
Incremental class: New Valid output dimension: 19
cats =   <torch.utils.data.dataloader.DataLoader object at 0x00000183601D2130>  <torch.utils.data.dataloader.DataLoader object at 0x00000183601D2340>  FER
Epoch:0
LR: 0.0001
Itr		Time		  Data		  Loss		Acc


d:\anaconda\envs\TORCHGPU\lib\site-packages\torch\optim\lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[0/67]	2.3436 (2.3436)	0.0102 (0.0102)	1.861 (1.861)	0.00 (0.00)
[66/67]	1.0531 (1.2767)	0.0030 (0.0029)	0.874 (1.354)	68.75 (52.12)
 * Train Acc 52.125
f1 score: 0.41686009538950713
 * Val Acc 57.250, Total time 9.63
validation split name: AU
f1 score: 0.23115079365079363
 * Val Acc 28.755, Total time 5.81
validation split name: FER
f1 score: 0.41686009538950713
 * Val Acc 57.250, Total time 9.10
The Accuracy table is  OrderedDict([('AU', {'AU': 33.047210300429185}), ('FER', {'AU': 28.755364806866954, 'FER': 57.25})])
Task AU average acc: 33.047210300429185
Task FER average acc: 43.002682403433475
===Summary of experiment repeats: 1 / 1 ===
The regularization coefficient: 100.0
The last avg acc of all repeats: [43.0026824]
mean: 43.002682403433475 std: 0.0
reg_coef: 1.0 mean: 42.877682403433475 std: 0.0
reg_coef: 10.0 mean: 47.2945278969957 std: 0.0
reg_coef: 100.0 mean: 43.002682403433475 std: 0.0
